# Body Language Decoder

## Table of Content
0. [Import Dependencies](#install)
1. [Detection using MediaPipe](#detection) 
2. [Feature Extraction](#feature-extraction) 
    1. [Write Columns Head in CSV File](#csv-header)
    2. [Extract Features of Assigned Class](#save-coordinates)
3. [Train Custom Model Using Scikit Learn](#model)
    1. [Load and Preprocess Input Data](#load-input)
    2. [Train Machine Learning Classification Models](#training)
    3. [Evaluate and Serialize Model](#evaluate)
4. [Real-time Detections with Model](#real-time-detection)

## 0. Import Dependencies <a id="install"></a>

In [1]:
from mediapipe import solutions as mp
import cv2
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
import os

## 1. Detection using MediaPipe <a id="detection"></a>

In [2]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)

        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))

        cv2.imshow("Holistic Model Detection", image)
        
        # Press "q" to exit
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [3]:
results.pose_landmarks

landmark {
  x: 0.61304224
  y: 0.55787224
  z: -1.4244294
  visibility: 0.9999051
}
landmark {
  x: 0.6298198
  y: 0.47542098
  z: -1.3517445
  visibility: 0.99986625
}
landmark {
  x: 0.6508764
  y: 0.4738746
  z: -1.3512311
  visibility: 0.9997867
}
landmark {
  x: 0.67394483
  y: 0.47262228
  z: -1.3515213
  visibility: 0.99979836
}
landmark {
  x: 0.56611264
  y: 0.47953436
  z: -1.3717101
  visibility: 0.99989295
}
landmark {
  x: 0.53901947
  y: 0.48100752
  z: -1.3727593
  visibility: 0.9998404
}
landmark {
  x: 0.5122797
  y: 0.483431
  z: -1.3736737
  visibility: 0.99989134
}
landmark {
  x: 0.69210017
  y: 0.50202566
  z: -0.8076345
  visibility: 0.9998362
}
landmark {
  x: 0.4659388
  y: 0.5235263
  z: -0.8976943
  visibility: 0.99991626
}
landmark {
  x: 0.6477269
  y: 0.645585
  z: -1.2025502
  visibility: 0.99976474
}
landmark {
  x: 0.55868226
  y: 0.65308213
  z: -1.2317512
  visibility: 0.9999168
}
landmark {
  x: 0.7812781
  y: 0.8341467
  z: -0.38417652
  visibility

## 2. Feature Extraction <a id="feature-extraction"></a>

### Write Columns Head in CSV File <a id="csv-header"></a>

In [4]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [5]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),
                  'z{}'.format(val), 'v{}'.format(val)]

In [6]:
if not os.path.exists("data"):
    os.mkdir("data")

with open("data/body_language_coords.csv", mode="w", newline="" ) as f:
    csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Extract Features of Assigned Class <a id="save-coordinates"></a>

- Repeat this step to save features of different targets.
- Make sure your samples cover different scenario of the target.

In [13]:
class_name = "raise"

In [14]:
cap = cv2.VideoCapture(0)
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break
            
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)

        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))
        
        # Export coordinates
        if results.pose_landmarks and results.face_landmarks:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in pose]).flatten())

                
            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in face]).flatten())
            
#             # Extract right hand landmarks
#             right_hand = results.right_hand_landmarks.landmark
#             right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
#                                             for landmark in right_hand]).flatten())
           
            # Concate row
            row = pose_row + face_row 
            
            # Append class name
            row.insert(0, class_name)
            
            # Export to CVS
            with open("data/body_language_coords.csv", mode="a", newline="" ) as f:
                csv_writer = csv.writer(f, delimiter=",", quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        
        cv2.imshow("Holistic Model Detection", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 3. Train Custom Model Using Scikit Learn <a id="model"></a>

### Load and Preprocess Input Data <a id="load-input"></a>

In [15]:
df = pd.read_csv("data/body_language_coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,raise,0.593186,0.598809,-1.213561,0.999904,0.625469,0.514669,-1.128540,0.999866,0.646285,...,-0.007067,0.0,0.672212,0.486739,0.018728,0.0,0.678413,0.477258,0.019519,0.0
1,raise,0.593851,0.591147,-1.247330,0.999897,0.624886,0.506280,-1.160838,0.999860,0.646274,...,-0.007463,0.0,0.667850,0.482251,0.016979,0.0,0.673995,0.473525,0.017612,0.0
2,raise,0.593853,0.585720,-1.278898,0.999890,0.623644,0.501890,-1.194330,0.999850,0.646027,...,-0.009104,0.0,0.666905,0.480770,0.016887,0.0,0.672814,0.472146,0.017672,0.0
3,raise,0.594156,0.580311,-1.268511,0.999875,0.623393,0.498189,-1.184703,0.999817,0.646017,...,-0.006897,0.0,0.669075,0.479277,0.018313,0.0,0.674947,0.470278,0.019120,0.0
4,raise,0.595194,0.579161,-1.281266,0.999870,0.623477,0.495866,-1.197191,0.999815,0.646341,...,-0.006345,0.0,0.669127,0.479864,0.019173,0.0,0.675245,0.470563,0.020041,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,cross,0.577525,0.638452,-1.315247,0.999937,0.604702,0.556067,-1.252681,0.999780,0.623099,...,-0.011576,0.0,0.660247,0.547654,0.014213,0.0,0.664878,0.540392,0.015213,0.0
1086,cross,0.583287,0.645594,-1.477063,0.999933,0.606806,0.561431,-1.428070,0.999765,0.624593,...,-0.007374,0.0,0.660836,0.564969,0.017807,0.0,0.666700,0.557864,0.018827,0.0
1087,cross,0.606390,0.626014,-1.437074,0.999880,0.631299,0.545856,-1.407070,0.999641,0.650031,...,-0.008728,0.0,0.688424,0.539026,0.024419,0.0,0.695402,0.526990,0.026245,0.0
1088,cross,0.613654,0.622132,-1.400073,0.999863,0.639261,0.541899,-1.361269,0.999651,0.661319,...,-0.011687,0.0,0.697798,0.538586,0.013705,0.0,0.703860,0.526798,0.015053,0.0


In [16]:
X = df.drop("class", axis=1)
y = df["class"]

In [17]:
y.value_counts()

cross          441
left cross     290
raise          255
right cross    104
Name: class, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

### Train Machine Learning Classification Models <a id="training"></a>

In [19]:
pipelines = {
    "lr":make_pipeline(StandardScaler(), LogisticRegression()),
    "svc":make_pipeline(StandardScaler(), SVC()),
    "rf":make_pipeline(StandardScaler(), RandomForestClassifier()),
    "knn":make_pipeline(StandardScaler(), KNeighborsClassifier()),
}

In [20]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train.ravel())
    fit_models[algo] = model

C:\Users\ADMIN\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluate and Serialize Model <a id="evaluate"></a>

In [21]:
# Show accuracy
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, pred))

lr 1.0
svc 0.9954128440366973
rf 1.0
knn 0.9862385321100917


In [22]:
# Show confusion matrix
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, confusion_matrix(y_test, pred), sep="\n", end="\n\n")

lr
[[87  0  0  0]
 [ 0 62  0  0]
 [ 0  0 47  0]
 [ 0  0  0 22]]

svc
[[87  0  0  0]
 [ 1 61  0  0]
 [ 0  0 47  0]
 [ 0  0  0 22]]

rf
[[87  0  0  0]
 [ 0 62  0  0]
 [ 0  0 47  0]
 [ 0  0  0 22]]

knn
[[87  0  0  0]
 [ 2 59  1  0]
 [ 0  0 47  0]
 [ 0  0  0 22]]



In [23]:
# Create folder if it does not exist
if not os.path.exists("generated_model"):
    os.mkdir("generated_model")
    
# Save model to file
model_to_save = "rf"

with open("generated_model/body_language_model.pkl", "wb") as f:
    pickle.dump(fit_models[model_to_save], f)

## 4. Real-time Detections with Model <a id="real-time-detection"></a>

In [25]:
with open("generated_model/body_language_model.pkl", "rb") as f:
    model_inference = pickle.load(f)

In [26]:
cap = cv2.VideoCapture(0)
with mp.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            break

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)


        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks
        mp.drawing_utils.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(242, 216, 175), thickness=1, circle_radius=1),
                                        mp.drawing_utils.DrawingSpec(color=(242, 216, 175), thickness=1))

        # Right hand
        mp.drawing_utils.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Left hand
        mp.drawing_utils.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(181, 137, 82), thickness=2, circle_radius=4),
                                        mp.drawing_utils.DrawingSpec(color=(181, 137, 82), thickness=2))

        # Pose 
        mp.drawing_utils.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS,
                                        mp.drawing_utils.DrawingSpec(
                                            color=(65, 53, 31), thickness=2, circle_radius=2),
                                        mp.drawing_utils.DrawingSpec(color=(65, 53, 31), thickness=2))
        
        # Export coordinates
        if results.pose_landmarks and results.face_landmarks:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in pose]).flatten())

                
            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                      for landmark in face]).flatten())
           
            # Concate row
            row = pose_row + face_row 
            
            # Predict using inference model
            X = pd.DataFrame([row])
            pred = model_inference.predict(X)[0]
            prob = np.max(model_inference.predict_proba(X)[0]).round(2)
            print(pred, prob)
            
            # Display result
            cv2.rectangle(image, (0,0), (250,60), (245, 117, 16), -1)
            cv2.putText(image, "CLASS", (95,12), cv2.FONT_HERSHEY_SIMPLEX,
                       0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, pred, (90,40), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, "PROB", (15,12), cv2.FONT_HERSHEY_SIMPLEX,
                       0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(prob), (10,40), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.imshow("Holistic Model Detection", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

left cross 0.54
right cross 0.59
right cross 0.62
right cross 0.4
right cross 0.38
right cross 0.47
right cross 0.5
right cross 0.51
cross 0.4
cross 0.57
left cross 0.61
left cross 0.68
left cross 0.78
left cross 0.86
left cross 0.88
left cross 0.93
left cross 0.91
left cross 0.99
left cross 0.97
left cross 0.95
left cross 0.93
left cross 0.67
cross 0.51
cross 0.5
cross 0.57
cross 0.61
cross 0.71
cross 0.77
cross 0.69
cross 0.69
cross 0.7
cross 0.41
raise 0.49
raise 0.48
raise 0.5
raise 0.52
raise 0.53
raise 0.6
raise 0.6
raise 0.71
raise 0.67
raise 0.57
raise 0.74
raise 0.72
raise 0.7
raise 0.75
raise 0.75
raise 0.59
raise 0.49
